### With pipeline code

In [24]:
import pandas as pd
import numbers as np


# Sklearn-libraries
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2

In [25]:
url = "https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day19-understanding-your-data-descriptive-stats/train.csv"
df = pd.read_csv(url)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
# drop unnecassary column
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [27]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [28]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [29]:
# step 1  -> train/test/split

X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Embarked']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [31]:
y_train.sample(5)

58     S
765    S
184    S
677    S
304    S
Name: Embarked, dtype: object

In [32]:
# Imputation transformation

from math import remainder


trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
],remainder='passthrough')

In [33]:
# one hot encoding
trf2 = ColumnTransformer([
         ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1,6])
],remainder='passthrough')

In [34]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))
])

In [35]:
# Feature selection
trf4 = SelectKBest(score_func=chi2, k=8)

In [36]:
# train the model
trf5 =DecisionTreeClassifier()

### Create Pipeline

In [37]:
pipe = Pipeline([
    ('trf1', trf1), 
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4), 
    ('trf5', trf5)
])

### Pipeline Vs make_pipeline 
- Pipeline requires naming of steps, make_pipeline does not
- (Same applies to columnTransformer vs make_column_transformer)

In [38]:
# Alternate Syntax
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [39]:
# train
pipe.fit(X_train, y_train)

ValueError: Input contains NaN